In [3]:
pip install requests -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

def get_movie_info(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"请求出错: {e}")
        return None

def parse_movie_data(html_content):
    if not html_content:
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    movie_list = []
    
    for item in soup.select('div.item')[:10]:
        try:
            rank = item.select_one('div.pic em').text.strip()
            title = item.select_one('span.title').text.strip()
            rating = item.select_one('span.rating_num').text.strip()
            quote = item.select_one('span.inq')
            quote = quote.text.strip() if quote else "暂无评价"
            
            movie_list.append({
                '排名': rank,
                '片名': title,
                '评分': rating,
                '评语': quote
            })
        except AttributeError:
            continue
    
    return movie_list

def save_to_excel(data, filename='豆瓣电影Top10.xlsx'):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"数据已保存到 {filename}")

if __name__ == "__main__":
    url = 'https://movie.douban.com/top250'
    print("开始爬取豆瓣电影排行榜数据...")
    
    html_content = get_movie_info(url)
    movie_data = parse_movie_data(html_content)
    
    if movie_data:
        print(f"成功获取 {len(movie_data)} 条电影数据")
        for movie in movie_data:
            print(f"{movie['排名']}. {movie['片名']} - 评分: {movie['评分']} - 评语: {movie['评语']}")
        
        save_to_excel(movie_data)
    else:
        print("未能获取到电影数据")
    
    print("爬取完成")    

开始爬取豆瓣电影排行榜数据...
成功获取 10 条电影数据
1. 肖申克的救赎 - 评分: 9.7 - 评语: 暂无评价
2. 霸王别姬 - 评分: 9.6 - 评语: 暂无评价
3. 泰坦尼克号 - 评分: 9.5 - 评语: 暂无评价
4. 阿甘正传 - 评分: 9.5 - 评语: 暂无评价
5. 千与千寻 - 评分: 9.4 - 评语: 暂无评价
6. 美丽人生 - 评分: 9.5 - 评语: 暂无评价
7. 这个杀手不太冷 - 评分: 9.4 - 评语: 暂无评价
8. 星际穿越 - 评分: 9.4 - 评语: 暂无评价
9. 盗梦空间 - 评分: 9.4 - 评语: 暂无评价
10. 楚门的世界 - 评分: 9.4 - 评语: 暂无评价
数据已保存到 豆瓣电影Top10.xlsx
爬取完成


In [7]:
pip install lxml -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
from lxml import etree
import os
import time

def download_images():
    # 设置请求头，模拟浏览器访问
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    base_url = "http://pic.netbian.com/"
    
    try:
        # 发送请求获取网页内容
        response = requests.get(base_url, headers=headers)
        response.encoding = "gbk"
        html_content = response.text
        
        # 解析HTML内容
        tree = etree.HTML(html_content)
        
        # 提取图片URL列表
        img_urls = tree.xpath("//ul[@class='clearfix']/li/a/span/img/@src")
        
        if not img_urls:
            print("未找到图片URL")
            return
        
        print(f"找到 {len(img_urls)} 张图片")
        
        # 创建保存目录
        save_dir = "d:/images"
        os.makedirs(save_dir, exist_ok=True)
        
        # 下载每张图片
        for i, img_path in enumerate(img_urls):
            # 处理相对URL
            if img_path.startswith('/'):
                img_url = base_url + img_path[1:]  # 移除开头的斜杠
            else:
                img_url = base_url + img_path
            
            try:
                # 下载图片
                print(f"正在下载第 {i+1}/{len(img_urls)} 张图片: {img_url}")
                img_response = requests.get(img_url, headers=headers, timeout=10)
                img_response.raise_for_status()
                
                # 确定文件扩展名
                file_ext = os.path.splitext(img_url)[1] or '.jpg'
                
                # 保存图片
                save_path = os.path.join(save_dir, f"image_{i}{file_ext}")
                with open(save_path, 'wb') as f:
                    f.write(img_response.content)
                
                print(f"图片已保存至: {save_path}")
                
                # 添加延时，避免频繁请求
                time.sleep(0.5)
                
            except requests.exceptions.RequestException as e:
                print(f"下载图片失败: {e}")
                continue
        
        print(f"全部下载完成，共下载 {len(img_urls)} 张图片")
        
    except Exception as e:
        print(f"发生错误: {e}")

if __name__ == "__main__":
    download_images()    

找到 20 张图片
正在下载第 1/20 张图片: http://pic.netbian.com/uploads/allimg/250627/202545-175102714571ed.jpg
图片已保存至: d:/images\image_0.jpg
正在下载第 2/20 张图片: http://pic.netbian.com/uploads/allimg/250626/231909-17509511496e7b.jpg
图片已保存至: d:/images\image_1.jpg
正在下载第 3/20 张图片: http://pic.netbian.com/uploads/allimg/250627/172845-1751016525523a.jpg
图片已保存至: d:/images\image_2.jpg
正在下载第 4/20 张图片: http://pic.netbian.com/uploads/allimg/250627/170634-17510151946096.jpg
图片已保存至: d:/images\image_3.jpg
正在下载第 5/20 张图片: http://pic.netbian.com/uploads/allimg/250627/105030-17509926303a62.jpg
图片已保存至: d:/images\image_4.jpg
正在下载第 6/20 张图片: http://pic.netbian.com/uploads/allimg/250626/110455-1750907095c440.jpg
图片已保存至: d:/images\image_5.jpg
正在下载第 7/20 张图片: http://pic.netbian.com/uploads/allimg/250625/233227-17508655473d8e.jpg
图片已保存至: d:/images\image_6.jpg
正在下载第 8/20 张图片: http://pic.netbian.com/uploads/allimg/250614/201508-17499033082071.jpg
图片已保存至: d:/images\image_7.jpg
正在下载第 9/20 张图片: http://pic.netbian.com/uploads/allimg/

In [41]:
import time
import random
import json
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

class ScholarBibtexCrawler:
    def __init__(self):
        # 配置Chrome浏览器
        self.chrome_options = Options()
        self.chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        self.chrome_options.add_argument("--disable-notifications")
        self.chrome_options.add_argument("--start-maximized")
        self.chrome_options.add_argument("--incognito")
        self.chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.chrome_options.add_experimental_option('useAutomationExtension', False)
        
        # 模拟不同的用户代理
        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0'
        ]
        self.chrome_options.add_argument(f'user-agent={random.choice(user_agents)}')
        
        # 初始化浏览器
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.chrome_options
        )
        
        # 设置随机延时范围(秒)
        self.min_delay = 3
        self.max_delay = 8
        
        # 创建保存目录
        self.save_dir = "bibtex_results"
        os.makedirs(self.save_dir, exist_ok=True)
    
    def random_delay(self):
        """随机延时，模拟人类浏览行为"""
        delay = random.uniform(self.min_delay, self.max_delay)
        time.sleep(delay)
    
    def search_scholar(self, query):
        """在谷歌学术中搜索文献"""
        print(f"正在搜索: {query[:50]}...")
        self.driver.get("https://scholar.google.com/")
        
        # 等待页面加载
        self.random_delay()
        
        # 输入搜索关键词
        search_box = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "q"))
        )
        search_box.clear()
        
        # 模拟人类输入
        for char in query:
            search_box.send_keys(char)
            time.sleep(random.uniform(0.1, 0.3))
        
        search_box.send_keys(Keys.RETURN)
        
        # 等待搜索结果
        self.random_delay()
        
        # 检查是否有验证码
        if "sorry" in self.driver.current_url.lower():
            print("遇到验证码，请手动处理并按Enter继续...")
            input()
    
    def get_bibtex(self, query):
        """获取文献的BibTeX格式"""
        self.search_scholar(query)
        
        try:
            # 查找BibTeX按钮
            bibtex_buttons = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.gs_or_cit.gs_or_btn.gs_nph'))
            )
            
            if not bibtex_buttons:
                print(f"未找到文献: {query[:50]}...")
                return None
            
            # 点击第一个结果的BibTeX按钮
            bibtex_buttons[0].click()
            self.random_delay()
            
            # 等待BibTeX弹出窗口加载
            bibtex_content = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'pre'))
            ).text
            
            print(f"成功获取BibTeX格式")
            return bibtex_content
            
        except Exception as e:
            print(f"获取BibTeX失败: {e}")
            return None
    
    def process_queries(self, queries):
        """处理多个查询"""
        results = {}
        
        for i, query in enumerate(queries, 1):
            print(f"\n=== 处理第 {i}/{len(queries)} 个查询 ===")
            bibtex = self.get_bibtex(query)
            
            if bibtex:
                results[query] = bibtex
                
                # 保存到文件
                safe_filename = re.sub(r'[\\/*?:"<>|]', '_', query[:50])
                with open(f"{self.save_dir}/{safe_filename}.bib", 'w', encoding='utf-8') as f:
                    f.write(bibtex)
            
            # 增加随机延时，避免频繁请求
            self.random_delay()
            self.random_delay()  # 额外延时，降低请求频率
        
        # 保存所有结果到JSON文件
        with open(f"{self.save_dir}/all_results.json", 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return results
    
    def close(self):
        """关闭浏览器"""
        self.driver.quit()

if __name__ == "__main__":
    # 待查询的文献列表
    queries = [
        "Automatic crater detection and age estimation for mare regions on the lunar surface",
        "The origin of planetary impactors in the inner solar system",
        "Deep learning based systems for crater detection: A review",
        "A preliminary study of classification method on lunar topography and landforms",
        "The CosmoQuest Moon mappers community science project: The effect of incidence angle on the Lunar surface crater distribution"
    ]
    
    # 初始化爬虫
    crawler = ScholarBibtexCrawler()
    
    try:
        # 处理查询
        results = crawler.process_queries(queries)
        
        # 打印结果统计
        print(f"\n=== 爬取完成 ===")
        print(f"成功获取 {len(results)}/{len(queries)} 篇文献的BibTeX格式")
        
    finally:
        # 关闭浏览器
        crawler.close()    


=== 处理第 1/5 个查询 ===
正在搜索: Automatic crater detection and age estimation for ...
获取BibTeX失败: Message: 
Stacktrace:
	GetHandleVerifier [0x0xfa3b03+62899]
	GetHandleVerifier [0x0xfa3b44+62964]
	(No symbol) [0x0xdd10f3]
	(No symbol) [0x0xe1980e]
	(No symbol) [0x0xe19bab]
	(No symbol) [0x0xe625c2]
	(No symbol) [0x0xe3e554]
	(No symbol) [0x0xe5fd81]
	(No symbol) [0x0xe3e306]
	(No symbol) [0x0xe0d670]
	(No symbol) [0x0xe0e4e4]
	GetHandleVerifier [0x0x1204793+2556483]
	GetHandleVerifier [0x0x11ffd02+2537394]
	GetHandleVerifier [0x0xfca2fa+220586]
	GetHandleVerifier [0x0xfbaae8+157080]
	GetHandleVerifier [0x0xfc141d+184013]
	GetHandleVerifier [0x0xfaba68+95512]
	GetHandleVerifier [0x0xfabc10+95936]
	GetHandleVerifier [0x0xf96b5a+9738]
	BaseThreadInitThunk [0x0x76ae5d49+25]
	RtlInitializeExceptionChain [0x0x779cd09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x779cd021+561]


=== 处理第 2/5 个查询 ===
正在搜索: The origin of planetary impactors in the inner sol...
获取BibTeX失败: Message: 
Stacktrace:
	GetHa

In [51]:
import time
import random
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 文献列表
papers = [
    "Automatic crater detection and age estimation for mare regions on the lunar surface",
    "The origin of planetary impactors in the inner solar system",
    "Deep learning based systems for crater detection: A review",
    "A preliminary study of classification method on lunar topography and landforms",
    "The CosmoQuest Moon mappers community science project: The effect of incidence angle on the Lunar surface crater distribution",
    "Fast r-cnn",
    "You only look once: Unified, real-time object detection",
    "Attention is all you need",
    "End-to-end object detection with transformers"
]


def setup_driver():
    """配置并启动Chrome浏览器"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 无界面模式
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # 反自动化设置
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    # 设置真实用户代理
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    ]
    chrome_options.add_argument(f'user-agent={random.choice(user_agents)}')

    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # 隐藏自动化特征
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )

    return driver


def human_like_delay(min=0.5, max=3.0):
    """模拟人类操作延迟"""
    time.sleep(random.uniform(min, max))


def get_bibtex(driver, paper_title):
    """获取并返回文献的BibTeX引用格式"""
    try:
        # 构建搜索URL
        encoded_title = urllib.parse.quote_plus(paper_title)
        url = f"https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q={encoded_title}"

        # 访问搜索页面
        driver.get(url)
        human_like_delay(1.0, 2.5)

        # 等待结果加载
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl"))
        )

        # 模拟人类滚动
        for _ in range(2):
            driver.execute_script("window.scrollBy(0, window.innerHeight * 0.5);")
            human_like_delay(0.7, 1.5)

        # 获取第一个结果
        first_result = driver.find_element(By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl")

        # 显示更多的引用选项
        cite_button = first_result.find_element(By.CSS_SELECTOR, "a.gs_or_cit")
        driver.execute_script("arguments[0].click();", cite_button)
        human_like_delay(1.0, 2.0)

        # 等待引用弹出框出现
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.gs_citr"))
        )

        # 切换到BibTeX格式
        bibtex_link = driver.find_element(By.XPATH, "//a[contains(., 'BibTeX')]")
        bibtex_url = bibtex_link.get_attribute('href')

        # 直接访问BibTeX链接
        driver.get(bibtex_url)
        human_like_delay(2.0, 3.0)

        # 获取并返回BibTeX内容
        pre_element = driver.find_element(By.TAG_NAME, 'pre')
        bibtex_content = pre_element.text
        return bibtex_content

    except (TimeoutException, NoSuchElementException):
        print(f"未找到文献: {paper_title[:50]}...")
        return f"@article{{ERROR: 未能找到文献引用 - {paper_title[:50]}...}}\n"
    except Exception as e:
        print(f"处理文献时出错: {paper_title[:50]}..., 原因: {str(e)[:100]}")
        return f"@article{{ERROR: {str(e)[:100].replace('@', '')}}}\n"


def main():
    driver = setup_driver()
    output = "以下是您文献的BibTeX引用格式：\n\n"

    try:
        # 首先访问主页建立会话
        driver.get("https://scholar.google.com")
        human_like_delay(2.0, 4.0)

        for i, paper in enumerate(papers, 1):
            print(f"正在处理第 {i}/{len(papers)} 篇文献: {paper[:50]}...")

            # 随机延迟避免封禁
            human_like_delay(random.uniform(3.0, 7.0))

            # 获取文献引用
            bibtex = get_bibtex(driver, paper)

            # 添加到输出
            output += f"%%% 文献 {i} - {paper[:60]}... %%%\n"
            output += bibtex + "\n\n"

            # 返回主页
            driver.get("https://scholar.google.com")
            human_like_delay(1.5, 2.5)

        # 显示最终结果
        print("\n" + "=" * 70)
        print(output)
        print("=" * 70)
        print("所有文献的BibTeX格式已成功生成！")

    except Exception as e:
        print(f"严重错误: {str(e)}")
    finally:
        driver.quit()
        print("浏览器已关闭")


if __name__ == "__main__":
    main()

正在处理第 1/9 篇文献: Automatic crater detection and age estimation for ...
正在处理第 2/9 篇文献: The origin of planetary impactors in the inner sol...
正在处理第 3/9 篇文献: Deep learning based systems for crater detection: ...
正在处理第 4/9 篇文献: A preliminary study of classification method on lu...
正在处理第 5/9 篇文献: The CosmoQuest Moon mappers community science proj...
未找到文献: The CosmoQuest Moon mappers community science proj...
正在处理第 6/9 篇文献: Fast r-cnn...
未找到文献: Fast r-cnn...
正在处理第 7/9 篇文献: You only look once: Unified, real-time object dete...
未找到文献: You only look once: Unified, real-time object dete...
正在处理第 8/9 篇文献: Attention is all you need...
未找到文献: Attention is all you need...
正在处理第 9/9 篇文献: End-to-end object detection with transformers...
未找到文献: End-to-end object detection with transformers...

以下是您文献的BibTeX引用格式：

%%% 文献 1 - Automatic crater detection and age estimation for mare regio... %%%
@inproceedings{salih2017automatic,
  title={Automatic crater detection and age estimation for mare regions on the 

In [53]:
import time
import random
import re
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 文献列表
papers = [
    "Automatic crater detection and age estimation for mare regions on the lunar surface",
    "The origin of planetary impactors in the inner solar system",
    "Deep learning based systems for crater detection: A review",
    "A preliminary study of classification method on lunar topography and landforms",
    "The CosmoQuest Moon mappers community science project: The effect of incidence angle on the Lunar surface crater distribution",
    "Fast r-cnn",
    "You only look once: Unified, real-time object detection",
    "Attention is all you need",
    "End-to-end object detection with transformers"
]

def setup_driver():
    """配置并启动Chrome浏览器"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 无界面模式
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    # 反自动化设置
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # 设置真实用户代理
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    ]
    chrome_options.add_argument(f'user-agent={random.choice(user_agents)}')
    
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # 隐藏自动化特征
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )
    
    return driver

def human_like_delay(min=0.5, max=3.0):
    """模拟人类操作延迟"""
    time.sleep(random.uniform(min, max))

def get_bibtex(driver, paper_title):
    """获取并返回文献的BibTeX引用格式"""
    try:
        print(f"🔍 正在查找: {paper_title[:50]}...")
        
        # 构建搜索URL
        encoded_title = urllib.parse.quote_plus(paper_title)
        url = f"https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q={encoded_title}"
        
        # 访问搜索页面
        driver.get(url)
        human_like_delay(1.5, 3.0)
        
        # 检查是否有验证码
        if "sorry" in driver.current_url or "验证码" in driver.page_source:
            print("⚠️ 检测到验证码，请手动处理后按Enter继续...")
            input()
            return get_bibtex(driver, paper_title)
        
        # 等待结果加载
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl"))
            )
        except TimeoutException:
            # 尝试不同的选择器
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.gs_ri"))
            )
        
        # 模拟人类滚动
        for _ in range(2):
            driver.execute_script("window.scrollBy(0, window.innerHeight * 0.5);")
            human_like_delay(0.7, 1.5)
        
        # 获取第一个结果
        try:
            first_result = driver.find_element(By.CSS_SELECTOR, "div.gs_r.gs_or.gs_scl")
        except NoSuchElementException:
            first_result = driver.find_element(By.CSS_SELECTOR, "div.gs_ri")
        
        # 显示更多的引用选项 - 尝试多种定位方式
        try:
            cite_button = first_result.find_element(By.CSS_SELECTOR, "a.gs_or_cit")
        except NoSuchElementException:
            cite_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Cite')]"))
            )
        
        # 使用JS点击，更可靠
        driver.execute_script("arguments[0].click();", cite_button)
        human_like_delay(1.5, 3.0)
        
        # 等待引用弹出框出现
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.gs_citr"))
        )
        
        # 切换到BibTeX格式
        try:
            bibtex_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'BibTeX')]"))
            )
            bibtex_link.click()
        except Exception:
            # 如果直接点击链接失败，尝试获取链接并访问
            bibtex_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(., 'BibTeX')]"))
            )
            bibtex_url = bibtex_link.get_attribute('href')
            driver.get(bibtex_url)
        
        human_like_delay(2.0, 4.0)
        
        # 获取并返回BibTeX内容 - 尝试多种定位方式
        try:
            pre_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, 'pre'))
            )
        except TimeoutException:
            pre_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'code'))
            )
        
        bibtex_content = pre_element.text
        
        # 验证BibTeX格式是否有效
        if "@" not in bibtex_content:
            raise ValueError("获取的内容不是有效的BibTeX格式")
        
        print(f"✅ 成功获取 {paper_title[:30]}... 的BibTeX格式")
        return bibtex_content
        
    except (TimeoutException, NoSuchElementException) as e:
        print(f"🚫 未找到文献: {paper_title[:50]}..., 原因: {str(e)[:100]}")
        return f"@article{{ERROR: 未能找到文献 - {re.sub('[^a-zA-Z0-9]', '_', paper_title[:30])}}}\n"
    except Exception as e:
        print(f"🚫 处理文献时出错: {paper_title[:50]}..., 原因: {str(e)[:100]}")
        return f"@article{{ERROR: {re.sub('[^a-zA-Z0-9]', '_', str(e)[:30])}}}\n"

def main():
    driver = setup_driver()
    output = "以下是您文献的BibTeX引用格式：\n\n"
    success_count = 0
    
    try:
        # 首先访问主页建立会话
        driver.get("https://scholar.google.com")
        human_like_delay(2.0, 4.0)
        
        for i, paper in enumerate(papers, 1):
            print(f"\n📄 正在处理第 {i}/{len(papers)} 篇文献: {paper[:50]}...")
            
            # 随机延迟避免封禁
            human_like_delay(random.uniform(3.0, 7.0))
            
            # 获取文献引用
            bibtex = get_bibtex(driver, paper)
            
            # 添加到输出
            output += f"%%% 文献 {i} - {paper[:60]}... %%%\n"
            output += bibtex + "\n\n"
            
            # 统计成功数量
            if "@ERROR" not in bibtex:
                success_count += 1
            
            # 返回主页
            driver.get("https://scholar.google.com")
            human_like_delay(1.5, 2.5)
        
        # 显示最终结果
        print("\n" + "=" * 80)
        print(output)
        print("=" * 80)
        print(f"🎉 所有文献处理完成！成功获取 {success_count}/{len(papers)} 篇文献的BibTeX格式")
        
        # 保存到文件
        with open("literature_bibtex.txt", "w", encoding="utf-8") as f:
            f.write(output)
        print(f"💾 BibTeX格式已保存到 literature_bibtex.txt")
        
    except Exception as e:
        print(f"🚫 严重错误: {str(e)}")
    finally:
        driver.quit()
        print("浏览器已关闭")

if __name__ == "__main__":
    main()


📄 正在处理第 1/9 篇文献: Automatic crater detection and age estimation for ...
🔍 正在查找: Automatic crater detection and age estimation for ...
🚫 未找到文献: Automatic crater detection and age estimation for ..., 原因: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff62192cda5+78885]
	GetHandleVerifier [0x0x7ff62192ce

📄 正在处理第 2/9 篇文献: The origin of planetary impactors in the inner sol...
🔍 正在查找: The origin of planetary impactors in the inner sol...
🚫 未找到文献: The origin of planetary impactors in the inner sol..., 原因: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff62192cda5+78885]
	GetHandleVerifier [0x0x7ff62192ce

📄 正在处理第 3/9 篇文献: Deep learning based systems for crater detection: ...
🔍 正在查找: Deep learning based systems for crater detection: ...
🚫 未找到文献: Deep learning based systems for crater detection: ..., 原因: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff62192cda5+78885]
	GetHandleVerifier [0x0x7ff62192ce

📄 正在处理第 4/9 篇文献: A preliminary study of classification method on lu...
🔍 正在查找: A prelimina